In [1]:
# this one is used as a source of truth for calculating the peformance of the work


In [3]:
import pandas as pd
import numpy as np
import math 
import statistics
from pandas import Timestamp
import math 

In [51]:
# get stat for -old or new and then compare
def get_stat_for(file, authors, label='-new'):
    pd.set_option("display.max_columns", None)
    df = pd.read_csv(file)
 
    count = 0
    total = 0

    for (index, row) in df.iterrows():
        # calcualte total scores
        total_scores = {'history': 0, 'api': 0, 'code': 0, 'api_s': 0}

        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            c = row['code_at_' + str(i) + '_v']
            a = row['api_at_' + str(i) + '_v']

            if str(h) != 'nan':
                total_scores['history'] += h
            if str(c) != 'nan':
                total_scores['code'] += c
            if str(a) != 'nan':
                total_scores['api'] += a
                
        # calcualte and rank developer scores
        scores = {}        
        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            h_developer = row['history_at_' + str(i)]
            c = row['code_at_' + str(i) + '_v']
            c_developer = row['code_at_' + str(i)]
            a = row['api_at_' + str(i) + '_v']
            a_developer = row['api_at_' + str(i)]

            if str(h_developer) != 'nan':
                if str(h) == 'nan' or h == 0 or total_scores['history'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    scores[h_developer] = scores.get(h_developer, []) + [float(h * 100 / total_scores['history'])]

            if str(c_developer) != 'nan':
                if str(c) == 'nan' or c == 0 or total_scores['code'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    scores[c_developer] = scores.get(c_developer, []) + [ float(c * 100 / total_scores['code'])]


            if str(a_developer) != 'nan':
                if str(a) == 'nan' or a == 0 or total_scores['api'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    scores[a_developer] = scores.get(a_developer, []) + [row['jaccard_score'] * float(a * 100 / total_scores['api'])]
                    pass

                
        # calculate the total
        for index_, score in scores.items():
            if len(score) == 0:
                scores[index_] = 0
            else:
                scores[index_] = sum(score)  ## ** (1/float(len(score)))


        sort = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        chosen = ''
        if len(sort) != 0:
            chosen = sort[0][0]

        if chosen == row['assignees']:
            count += 1

            
        previous = row['assignees']
        total += 1

        if row['assignees'] not in authors:
            authors[row['assignees']] = {
                'total-new': 0, 'count_differently-new': 0, 'count-new': 0, 'count_history-new': 0, 'count_code-new': 0, 'count_api-new': 0,
                'total-old': 0, 'count_differently-old': 0, 'count-old': 0, 'count_history-old': 0, 'count_code-old': 0, 'count_api-old': 0,
                }

        authors[row['assignees']]['total' + label] += 1
        if row['assignees'] == row['history_at_1']:
            authors[row['assignees']]['count_history' + label] += 1
        if row['assignees'] == row['code_at_1']:
            authors[row['assignees']]['count_code' + label] += 1
        if row['assignees'] == row['api_at_1']:
            authors[row['assignees']]['count_api' + label] += 1
        if 1 == row['at_1']:
            authors[row['assignees']]['count' + label] += 1
        if 0 < len(sort) and sort[0][0] == row['assignees']:
            authors[row['assignees']]['count_differently' + label] += 1
            

        
        previous = row['chosen']
        previous_time = row['report_time']
        previous_component = row['component']
   

    print(label)
    print(count)
    print(count / total)
    print(total)
    return authors



authors_ = {}
authors_ = get_stat_for('../../data/output/jdt_direct.csv', authors_, '-new')
#authors_ = get_stat_for('../../data/output/jdt_direct-without-util.csv', authors_, '-old')

-new
284
0.47333333333333333
600
